**Importando bibliotecas**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

**Abrindo um arquivo CSV do drive**

In [2]:
base = pd.read_csv("../Bases/Criados/6_base_final.csv")

In [3]:
base

,Situação_censitária,V0031,Sexo,Cor/Raça,Plano_de_saúde,Frequência_distúrbios_do_sono,Frequência_falta_de_disposição,Frequência_falta_de_interesse,Frequência_sintomas_depressivos,P00901,...,Quantidade_de_trabalhos,Jornada_de_Trabalho,Exposição_poluentes,Atividade_fisica_semanal,Início_tabagismo,Tempo_parou_de_fumar,Intensidade_fumo,Doença_mental,Consumo_de_álcool,Faixa_IMC
0,1,1,2.0,3.0,2.0,1.0,1.0,1.0,1.0,2,...,1,1,2,0,0,0,0,2,0,3
1,1,1,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2,...,1,1,2,2,0,3,0,2,3,3
2,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2,...,1,1,2,0,1,3,0,2,2,2
3,1,1,2.0,3.0,2.0,1.0,1.0,2.0,1.0,3,...,0,0,2,3,0,0,0,2,0,2
4,1,1,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2,...,1,1,2,2,0,0,0,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20494,2,1,2.0,1.0,2.0,4.0,3.0,3.0,4.0,3,...,1,2,2,0,0,0,0,1,2,1
20495,2,1,1.0,2.0,2.0,2.0,3.0,3.0,3.0,2,...,1,2,1,0,0,0,0,2,0,1
20496,2,1,2.0,3.0,2.0,1.0,2.0,3.0,2.0,4,...,1,1,1,0,2,1,2,2,1,1
20497,2,1,1.0,3.0,2.0,2.0,1.0,1.0,1.0,4,...,1,1,1,0,3,1,2,2,3,2


**Verificando ausência na base de dados**

In [4]:
# índices das linhas que contém valores NaN
missing_indices = base[base.isnull().any(axis=1)].index

# imprime apenas as linhas com valoes ausentes
display(base.iloc[missing_indices])

,Situação_censitária,V0031,Sexo,Cor/Raça,Plano_de_saúde,Frequência_distúrbios_do_sono,Frequência_falta_de_disposição,Frequência_falta_de_interesse,Frequência_sintomas_depressivos,P00901,...,Quantidade_de_trabalhos,Jornada_de_Trabalho,Exposição_poluentes,Atividade_fisica_semanal,Início_tabagismo,Tempo_parou_de_fumar,Intensidade_fumo,Doença_mental,Consumo_de_álcool,Faixa_IMC


**Verificar se existem amostras inconsistentes e redundantes**

In [5]:
Nome_das_Colunas = base.columns[:-1]

In [6]:
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]
# se houver valores redundantes ou inconsistentes, imprima
if len(df_duplicates)>0:
    print('\nAmostras redundantes ou inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem valores duplicados')


Amostras redundantes ou inconsistentes:


,Situação_censitária,V0031,Sexo,Cor/Raça,Plano_de_saúde,Frequência_distúrbios_do_sono,Frequência_falta_de_disposição,Frequência_falta_de_interesse,Frequência_sintomas_depressivos,P00901,...,Quantidade_de_trabalhos,Jornada_de_Trabalho,Exposição_poluentes,Atividade_fisica_semanal,Início_tabagismo,Tempo_parou_de_fumar,Intensidade_fumo,Doença_mental,Consumo_de_álcool,Faixa_IMC
3811,1,1,1.0,3.0,2.0,1.0,1.0,1.0,1.0,4,...,1,1,2,3,0,0,0,2,0,2
4009,1,1,1.0,3.0,2.0,1.0,1.0,1.0,1.0,4,...,1,1,2,3,0,0,0,2,0,2


Como podemos ver, existem algumas amostras redundantes (duplicadas) e outras inconsistentes (amostras iguais, mas com classes distintas).
Primeiro, serão removidas as amostras redundantes, mantendo na base apenas uma delas.

In [7]:
def delDuplicatas( df_dataset ):
    '''
    Para cada grupo de amostras duplicadas, mantém uma e apaga as demais
    '''

    # remove as amostras duplicadas, mantendo apenas a primeira ocorrencia
    df_dataset = df_dataset.drop_duplicates(keep = 'first')

    return df_dataset

base = delDuplicatas( base )

**Após remover as amostras redundantes, é preciso checar se há amostras inconsistentes.**

In [8]:
# para detectar inconsistências, a rotina abaixo obtém as amostras onde os valores
# dos atributos continuam duplicados. Neste caso, os atributos serão iguais, mas as classes serão distintas
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]

# se tiver valores inconsistentes, imprime
if len(df_duplicates)>0:
    print('\nAmostras inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem mostras inconsistentes')


Não existem mostras inconsistentes


**Tendo amostras inconsistentes, será necessário apagar estas instâncias, já que não sabemos qual delas está correta**

In [9]:
def delInconsistencias(df_dataset):
    '''
    Remove todas as amostras inconsistentes da base de dados
    '''

    df_dataset = df_dataset.drop_duplicates(subset=Nome_das_Colunas,keep=False)

    return df_dataset

base = delInconsistencias(base)

# obtém apenas as amostras onde os valores dos atributos estão duplicados
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]

# se tiver valores redundantes ou inconsistentes, imprime
if len(df_duplicates)>0:
    display(df_duplicates)
else:
    print('Não existem amostras redundantes ou inconsistentes')


Não existem amostras redundantes ou inconsistentes
